## 建立電腦環境變數

In [ ]:
#建立電腦環境變數(模擬的)，並載入模擬環境
from dotenv import load_dotenv
load_dotenv()

## 連線資料庫

In [3]:
import psycopg2
import os
conn = psycopg2.connect(os.environ['postgre_password'])
conn.commit()
conn.close()

## 使用 with as 語法可以自動 commit(), close() conn or cursor 

In [6]:
#手動 commit(), close() conn
with psycopg2.connect(os.environ['postgre_password']) as conn:
    cusor = conn.cursor()
    sql = '''
    CREATE TABLE IF NOT EXISTS 客戶(
	客戶_id SERIAL,
	客戶名稱 VARCHAR(255) NOT NULL,
	PRIMARY KEY(客戶_id)
    );
'''
    cusor.execute(sql)

    sql = '''
    CREATE TABLE IF NOT EXISTS 聯絡人(
	聯絡人id SERIAL,
	客戶id INT,
	聯絡人姓名 VARCHAR(255) NOT NULL,
	電話 VARCHAR(15),
	郵件 VARCHAR(100),
	PRIMARY KEY(聯絡人id),
	FOREIGN KEY(客戶id) REFERENCES 客戶(客戶_id)
    );
'''
    cusor.execute(sql)
    cusor.close()

In [5]:
#半自動
conn = psycopg2.connect(os.environ['postgre_password'])
with conn:
    with conn.cursor()  as cursor:
        sql = '''
        INSERT INTO 客戶(客戶名稱)
        VALUES('遠傳電信'),('台灣大車隊');
        '''
        cursor.execute(sql)
conn.close()

In [7]:
#auto commit() & close()
with psycopg2.connect(os.environ['postgre_password']) as conn:
    with conn.cursor()  as cursor:
        sql = '''
        INSERT INTO 聯絡人(客戶id,聯絡人姓名,電話,郵件)
        VALUES (1,'John Doe','(408)-111-1234','john.doe@bluebird.dev'),
	    (1,'Jane Doe','(408)-111-1235','jane.doe@bluebird.dev'),
	    (2,'David Wright','(408)-222-1234','david.wright@dolphin.dev');
        '''
        cursor.execute(sql)

## 從DB取資料

In [11]:
with psycopg2.connect(os.environ['postgre_password']) as conn:
    with conn.cursor()  as cursor:
        sql = '''
        SELECT 聯絡人id,客戶名稱,聯絡人姓名,電話,郵件
        FROM 聯絡人 LEFT JOIN 客戶 ON 聯絡人.客戶id = 客戶.客戶_id
        '''
        cursor.execute(sql)
        datas:list = cursor.fetchmany(10)

for item in datas:
    print(item)
        

(1, '遠傳電信', 'John Doe', '(408)-111-1234', 'john.doe@bluebird.dev')
(2, '遠傳電信', 'Jane Doe', '(408)-111-1235', 'jane.doe@bluebird.dev')
(3, '台灣大車隊', 'David Wright', '(408)-222-1234', 'david.wright@dolphin.dev')


In [12]:
#不知道有幾筆資料時
with psycopg2.connect(os.environ['postgre_password']) as conn:
    with conn.cursor()  as cursor:
        sql = '''
        SELECT 聯絡人id,客戶名稱,聯絡人姓名,電話,郵件
        FROM 聯絡人 LEFT JOIN 客戶 ON 聯絡人.客戶id = 客戶.客戶_id
        '''
        cursor.execute(sql)
        while True:
            oneItem = cursor.fetchone()
            if oneItem is not None:
                print(oneItem)
            else:
                break

(1, '遠傳電信', 'John Doe', '(408)-111-1234', 'john.doe@bluebird.dev')
(2, '遠傳電信', 'Jane Doe', '(408)-111-1235', 'jane.doe@bluebird.dev')
(3, '台灣大車隊', 'David Wright', '(408)-222-1234', 'david.wright@dolphin.dev')
